In [7]:
import numpy as np
import cv2

from utilis.resize_label import *

In [10]:
# @staticmethod
def writePlyFile(file_name, vertices, colors):
    ply_header = '''ply
                format ascii 1.0
                element vertex %(vert_num)d
                property float x
                property float y
                property float z
                property uchar red
                property uchar green
                property uchar blue
                end_header
               '''
    vertices = vertices.reshape(-1, 3)
    colors = colors.reshape(-1, 3)
    vertices = np.hstack([vertices, colors])
    with open(file_name, 'w') as f:
        f.write(ply_header % dict(vert_num=len(vertices)))
        np.savetxt(f, vertices, '%f %f %f %d %d %d')

# Load data

In [2]:
coord_data_dir = "/data/cornucopia/ib255/derivative_datasets/cued_scene_coordinate_regression/data_from_jason/DS_003_JDB-Full/coordinates/"

In [5]:
data_size = 1
images = np.zeros((data_size, 2048, 4096, 3), dtype=np.int16)
labels = np.zeros((data_size, 2048, 4096, 3), dtype=np.float32)


for data in range(1,data_size + 1):
    if data < 10:
        data = "00" + str(data)
    elif data < 100:
        data = "0" + str(data)

    coord_npz = "0{}_rendered.png_config.npz".format(str(data))
    npz_data = np.load(coord_data_dir + coord_npz)
    
    images[int(data)-1,:,:,:] = npz_data['image_colors']
    labels[int(data)-1,:,:,:] = npz_data['points_3d_camera']

In [8]:
images_low = np.zeros((data_size, 224, 224, 3), dtype=np.int16)
labels_low = np.zeros((data_size, 224, 224, 3), dtype=np.float32)

for ind in range(data_size):
    images_low[ind,:,:,:] = cv2.resize(images[ind,:,:,:],(224, 224), interpolation = cv2.INTER_CUBIC)
    
for ind in range(data_size):
    labels_low[ind,:,:,:] = resize_coordinates(labels[ind,:,:,:], new_dim=(224,224))

# write PLY file

In [11]:
writePlyFile('practice.ply', labels_low[0], images_low[0])

# read PLY